In [1]:
#import dependancieies 
import datetime
import backtrader as bt
import pandas as pd
import math
import matplotlib
import warnings
warnings.filterwarnings('ignore')


In [10]:

def start_test():
    startdate=datetime.datetime(2019, 1, 1)
    enddate=datetime.datetime(2021, 6, 1)
    tickers = ["GME","ADBE","SONY"]
    for ticker in tickers:
        run_test(ticker,startdate,enddate)

#creating the parameters for cerebro to feed data to adjust dates to your own buy date

def run_test(ticker,startdate,enddate):

    cerebro = bt.Cerebro()
    cerebro.addstrategy(Cross)
    cerebro.addwriter(bt.WriterFile, csv=True,out=f"{ticker}_trade_log.csv", rounding=2)
    cerebro.broker.set_cash(1000)

    feed = bt.feeds.YahooFinanceData(
        dataname=ticker,
        timeframe=bt.TimeFrame.Days,
        fromdate=startdate,
        todate=enddate,
        reverse=False,
    )
    # print(feed)
    cerebro.adddata(feed)
    print("starting portfolio value: %.2f" % cerebro.broker.getvalue())
    cerebro.run(stdstats=True)
    

    print("final portfolio value: %.2f" % cerebro.broker.getvalue())
    
    cerebro.plot(style='candle',iplot=False,)

#create strategy held object


class Cross(bt.Strategy):

    # set parameters to define fast and slow
    params = (
        ("fast", 6),
        ("slow", 13),
        ("order_percentage", 0.99),
        ("ticker", "stock"),
    )

    # define constractors
    def __init__(self):
        print("position size:", self.position.size)
        _name='Custom Cross Strategy'
        self.fast_moving_average = bt.ind.EMA(
            self.data.open, period=self.params.fast, plotname="fast moving average"
        )

        self.slow_moving_average = bt.ind.EMA(
            self.data.close, period=self.params.slow, plotname="slow moving average"
        )

        self.crossover = bt.ind.CrossOver(
            self.fast_moving_average, self.slow_moving_average
        )

    def log(self, txt, dt=None):
        """ Logging function for this strategy"""
        dt = dt or self.data.datetime[0]
        if isinstance(dt, float):
            dt = bt.num2date(dt)
        print("%s, %s" % (dt.date(), txt))

    def notify_order(self, order):
        """ Triggered upon changes to orders. """

        # Suppress notification if it is just a submitted order.
        if order.status == order.Submitted:
            return

        # Print out the date, security name, order number and status.
        dt, dn = self.datetime.date(), order.data._name
        type = "Buy" if order.isbuy() else "Sell"
        self.log(
            f"{order.data._name:<6} Order: {order.ref:3d}\tType: {type:<5}\tStatus"
            f" {order.getstatusname():<8} \t"
            f"Size: {order.created.size:9.4f} Price: {order.created.price:9.4f} "
            f"Position: {self.getposition(order.data).size}"
        )
        if order.status == order.Margin:
            return

        # Check if an order has been completed
        if order.status in [order.Completed]:
            self.log(
                f"{order.data._name:<6} {('BUY' if order.isbuy() else 'SELL'):<5} "
                # f"EXECUTED for: {dn} "
                f"Price: {order.executed.price:6.2f} "
                f"Cost: {order.executed.value:6.2f} "
                f"Comm: {order.executed.comm:4.2f} "
                f"Size: {order.created.size:9.4f} "
            )

    def notify_trade(self, trade):
        """Provides notification of closed trades."""
        if trade.isclosed:
            self.log(
                "{} Closed: PnL Gross {}, Net {},".format(
                    trade.data._name,
                    round(trade.pnl, 2),
                    round(trade.pnlcomm, 1),
                )
            )

    def next(self):
        if self.position.size == 0:
            if self.crossover > 0:
                amount_to_invest = self.params.order_percentage * self.broker.cash
                self.size = math.floor(amount_to_invest / self.data.close)

                self.log(
                    "Buy {} shares of {} at {}".format(
                        self.size,
                        self.params.ticker,
                        self.data.close[0],
                    )
                )
                self.buy(size=self.size)

        if self.position.size > 0:
            if self.crossover < 0:
                self.log(
                    "Sell {} shares of {} at {}".format(
                        self.size, self.params.ticker, self.data.close[0],
                    )
                )
                self.sell(size=self.size)


if __name__ == "__main__":
    start_test()



00 Size:    3.0000 
2020-08-13, Sell 3 shares of stock at 450.02
2020-08-14, ADBE   Order: 767	Type: Sell 	Status Accepted 	Size:   -3.0000 Price:  450.0200 Position: 0
2020-08-14, ADBE   Order: 767	Type: Sell 	Status Completed 	Size:   -3.0000 Price:  450.0200 Position: 0
2020-08-14, ADBE   SELL  Price: 451.39 Cost: 1344.78 Comm: 0.00 Size:   -3.0000 
2020-08-14, ADBE Closed: PnL Gross 9.39, Net 9.4,
2020-08-14, Buy 3 shares of stock at 447.6
2020-08-17, ADBE   Order: 768	Type: Buy  	Status Accepted 	Size:    3.0000 Price:  447.6000 Position: 3
2020-08-17, ADBE   Order: 768	Type: Buy  	Status Completed 	Size:    3.0000 Price:  447.6000 Position: 3
2020-08-17, ADBE   BUY   Price: 450.00 Cost: 1350.00 Comm: 0.00 Size:    3.0000 
2020-09-21, Sell 3 shares of stock at 475.64
2020-09-22, ADBE   Order: 769	Type: Sell 	Status Accepted 	Size:   -3.0000 Price:  475.6400 Position: 0
2020-09-22, ADBE   Order: 769	Type: Sell 	Status Completed 	Size:   -3.0000 Price:  475.6400 Position: 0
2020-09-

Solving environment: done

# All requested packages already installed.

